### 1. Definição do problema
> O objetivo deste trabalho é apresentar uma análise sobre os preços de produtos importados pelo Brasil, comparando os valores declarados numa operação de importação com os preços praticados no comércio internacional.
>Para isso, primeiramente, será realizado um levantamento de preços junto ao mercado mundial para um grupo de mercadorias de alto valor agregado.
> Em seguida, buscar-se-ão os preços desses mesmos produtos declarados em processos de importação de mercadorias.
> Por fim, far-se-á a comparação entre as médias desses valores, visando a uma conclusão quanto a indícios de fraudes - subfaturamento ou superfaturamento - nas operações de importação realizadas pelo importador nacional.

### 2. A coleta de dados
> Os datasets usados neste trabalho foram coletados de fontes diversas. 
> Definiu-se que os sites __www.aliexpress.com__ e __www.alibaba.com__, notórios fornecedores de produtos diversos, serão objeto de webscrapping realizado com a ferramenta de código aberto _Beautifull Soup_ e _Selenium_, as quais serão utilizadas para coletar uma gama de aproximadamente 1 mil registros de cada um desses sites.
> Para delimitar o escopo da análise, será definido apenas uma gama de produtos, que apresenta uma variação grande de preços: __display lcd para aparelhos celulares__.

__Bold por favor__ <br>
_Italico Olha isso_ <br>
$ a + bX$ <br>
__________ dfjlkaj dlkjf lkajkdj
> item 
>>item2
>>>item3
- item 1
- item 2
1. item 1
2. item 2 <br>
<font color=blue>"font color=blue"</font><br>
paragrafog <br> outro paragrafo<br>
#isto é comentário

In [1]:
from bs4 import BeautifulSoup
import csv
import pandas as pd

#bibliotecas para requisição de páginas dinâmicas
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import time
import sys
import re
    
def alteraIdioma():
    #Mudando o foco do mouse para a paǵina inicial
    user_id = driver.find_element_by_xpath('//*[@id="nav-user-account"]') 
    ActionChains(driver).move_to_element(user_id).click().perform()
    time.sleep(3)

    #Aqui começo a navegar para alterar o idioma
    driver.find_element_by_xpath('//*[@id="switcher-info"]').click()
    time.sleep(3)
    painel = driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div')
    painel.click()
    #painel country selector switcher
    time.sleep(3)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[1]/div/a[1]/span/span').click()
    time.sleep(3)
    painel.find_element_by_xpath("/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[1]/div/div[1]/ul/li[224]").click()
    time.sleep(1)
    driver.find_element_by_xpath('/html/body/div[1]/div[3]/div/div/div[4]/div/div/div/div[4]/button').click()
    driver.refresh()           

#Preparando a aquisição das páginas
#Criando informação sobre quem está fazendo o scrapping
headers = {
    'User-Agent': 'João Lima, from PUC-MG University',
    'From': 'jbetol@gmail.com'
}

#Ainda precisa Verificar com Try Except se driver para Firefox (o browser que escolhi)está disponível
try:
    driver = webdriver.Firefox('/media/joao/HDLinux/tools')
except IOError:
    sys.exit("Geckodriver not found!")

#Faremos o scrapping nos sites AliExpress e AliBaba, idioma inglês e dólar como moeda.   
url_aliexpress = 'https://www.aliexpress.com/wholesale?trafficChannel=main&d=y&CatId=0&SearchText=phone+display+lcd+for+screen&ltype=wholesale&SortType=default&page='

url_alibaba = 'https://www.alibaba.com/products/phone_display_lcd_screen/CID5904002.html?spm=a2700.galleryofferlist.0.0.126c1524tZVgH8&IndexArea=product_en&page='

paginas = []
qnt_paginas = 99
passou_aliexpress = False # Para controlar a mudança de idioma e moeda
alibaba = False # Vamos começar scrapping por AliExpress.com
site_index = 2  # números de sites a ser scrapping: AliExpress e Alibaba
descricao = []  #descrição das mercadorias tela de lcd para smartphone
preco_min = []
preco_max = []
baba_or_express = [] #Valor booleano para saber origem do item, se AliExpress oou Alibaba

#Vamos pegar 99 páginas de dados (no máximo) de cada site escolhido e jogar num array
while site_index > 0  :
    site_index = site_index - 1

    #Montando a lista de páginas
    for i in range(1,qnt_paginas):
        if not alibaba:
            url = url_aliexpress + str(i)
        else:
            url = url_alibaba + str(i)

        paginas.append(url)

    #Navegando nas páginas de aquisição de dados
    for pagina in paginas:
        driver.get(pagina)

        #O site AliExpress pode estar em português, então temos que alterar para inglês 
        if not alibaba and not passou:
            alteraIdioma()
            passou = True

        req = driver.page_source
        soup = BeautifulSoup(req, "html.parser")

    #Adquirindo a lista dos produtos que se encontra na classe indicada
        if alibaba:
            classes = soup.find('div', class_ = "organic-list app-organic-search__list")
            #Agora vamos buscar os itens da lista
            items =  classes.find_all(class_ = 'organic-gallery-offer-outter J-offer-wrapper')
        else:
            classes = soup.find('ul', class_ = "list-items")
            #Agora vamos buscar os itens da lista
            try:
                items =  classes.find_all('li', class_ = 'list-item')
            except AttributeError: # É possível que tenhamos menos páginas do que previsto
                break

        #Para cada item, pegar apenas descrição e o preco e guardar num dicionário
        for i,item in enumerate(items):
            if alibaba:
                descricao.append(item.find('h4').get('title'))
                preco = item.find('p', class_ = 'elements-offer-price-normal').get('title')
            else:
                descricao.append(item.find('a', class_ = 'item-title').get('title'))
                preco = item.find('span', class_ = 'price-current').text

            baba_or_express.append(alibaba)
            preco_min_max = re.findall(r'(\d+[\.,]\d{1,2})', preco)
            if len(preco_min_max) == 1:
                preco_min_max.insert(0,'0.00') 
            preco_min.append(preco_min_max[0])
            preco_max.append(preco_min_max[1])
            
    alibaba = True
    paginas.clear()

driver.quit()
#transformando o dicionário em DataFrame do Pandas
df = pd.DataFrame({
    'descricao': descricao,
    'preco_minimo': preco_min,
    'preco_maximo': preco_max,
    'alibaba': baba_or_express
    })
df.info()
# saving the DataFrame as a CSV file 
gfg_csv_data = df.to_csv('lista.csv', header = True) 




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   descricao     32 non-null     object
 1   preco_minimo  32 non-null     object
 2   preco_maximo  32 non-null     object
 3   alibaba       32 non-null     bool  
dtypes: bool(1), object(3)
memory usage: 928.0+ bytes


In [1]:
import re
dici = {}
fonte = '$10.63-$16.82' # - US $43.77 - 44.27 $15.00-$ 39.81 US $18.49 - US325,30 US $12,70'
fonte_limpa = re.findall(r'(\d+[\.,]\d{1,2})', fonte) #re.sub(r'[aA-Zz$\-\s+]','',fonte)
#fonte_limpa = re.sub(r'[aA-Zz]','',fonte_limpa)
dici = fonte,fonte_limpa[0], fonte_limpa[1]
print(dici)

#print(re.findall(r'(\d+[\.,]\d{1,2})', fonte))


('$10.63-$16.82', '10.63', '16.82')
